<a href="https://colab.research.google.com/github/zelalemamera-stonybrook/projects-sandbox/blob/main/Multi_Layer_Perceptron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np

In [113]:
X_train = np.load('/content/X_train.npy')
X_dev = np.load('/content/X_dev.npy')
Y_train = np.load('/content/Y_train.npy')
Y_dev = np.load('/content/Y_dev.npy')

In [114]:
print(X_train.shape, X_dev.shape, Y_train.shape, Y_dev.shape)

(412, 4200) (52, 4400) (412,) (52,)


In [115]:
X_train = torch.as_tensor(X_train, dtype = torch.float32)
X_dev = torch.as_tensor(X_dev, dtype = torch.float32)
Y_train = torch.add(torch.as_tensor(Y_train, dtype = torch.long), -1)
Y_dev = torch.add(torch.as_tensor(Y_dev, dtype = torch.long), -1)


In [116]:
a = torch.zeros((412, 200))
X_train = torch.cat((X_train, a), 1)

In [117]:
print(X_train.shape, X_dev.shape, Y_train.shape, Y_dev.shape)
print(X_train.dtype, X_dev.dtype, Y_train.dtype, Y_dev.dtype)

torch.Size([412, 4400]) torch.Size([52, 4400]) torch.Size([412]) torch.Size([52])
torch.float32 torch.float32 torch.int64 torch.int64


In [202]:

class MLP(nn.Module):
  '''
  '''
  def __init__(self, input_dimension, density1, density2, density3,
               density4, density5, output_dimension):
    super().__init__()
    self.layer1 = nn.Linear(input_dimension, density1)
    self.layer2 = nn.Linear(density1, density2)
    self.layer3 = nn.Linear(density2, density3)
    self.layer4 = nn.Linear(density3, density4)
    self.layer5 = nn.Linear(density4, density5)
    self.layer6 = nn.Linear(density5, output_dimension)
    self.activate = nn.ReLU()

  def forward(self, input_batch):
    softmax = nn.Softmax(dim=1)
    hidden1 = self.activate(self.layer1(input_batch))
    hidden2 = self.activate(self.layer2(hidden1))
    hidden3 = self.activate(self.layer3(hidden2))
    hidden4 = self.activate(self.layer4(hidden3))
    hidden5 = self.activate(self.layer5(hidden4))
    output_batch = softmax(self.layer6(hidden5))
    return output_batch


In [233]:
def train(model, training_data, target_data, max):
  '''
  '''
  model.train()
  loss_function = nn.CrossEntropyLoss()
  optimizer = torch.optim.SGD(model.parameters())
  accuracy = 0
  n = 0
  last_improvement = 0
  while(accuracy < 90 and n < max and last_improvement < 50):
    optimizer.zero_grad()
    loss_function(model.forward(training_data), target_data).backward()
    optimizer.step()
    accuracy_new = model_accuracy(model, training_data, target_data)
    if accuracy_new <= accuracy:
      last_improvement +=1
    else:
      last_improvement = 0
    accuracy = accuracy_new
    print('Score:', accuracy, '\nEpoch: ', n)
    n += 1


In [199]:
def model_accuracy(model, input_data, target_data):
  '''
  '''
  model.eval()
  predictions = torch.argmax(model.forward(input_data), dim=1)
  score = 100 * (torch.sum(torch.eq(predictions, target_data))).item() / len(target_data)
  return score


In [182]:
def confusion_matrix(model, input_data, target_data):
  '''
  '''
  predictions = torch.argmax(model.forward(input_data), dim=1)
  true_class1_model_class1 = 0
  true_class2_model_class2 = 0
  true_class3_model_class3 = 0
  true_class1_model_class2 = 0
  true_class1_model_class3 = 0
  true_class2_model_class1 = 0
  true_class2_model_class3 = 0
  true_class3_model_class1 = 0
  true_class3_model_class2 = 0
  for i in range(len(target_data)):
    if target_data[i].item() == predictions[i].item():
      c = target_data[i].item()
      if c == 0:
        true_class1_model_class1+=1
      elif c == 1:
        true_class2_model_class2+=1
      else:
        true_class3_model_class3+=1
    else:
      target = target_data[i].item()
      predicted = predictions[i].item()
      if target == 0:
        if predicted == 1:
          true_class1_model_class2+=1
        else:
          true_class1_model_class3+=1
      elif target == 1:
        if predicted == 0:
          true_class2_model_class1+=1
        else:
          true_class2_model_class3+=1
      else:
        if predicted == 0:
          true_class3_model_class1+=1
        else:
          true_class3_model_class2+=1
  confusion_matrix = torch.tensor(
        [[true_class1_model_class1, true_class1_model_class2, true_class1_model_class3],
        [true_class2_model_class1, true_class2_model_class2, true_class2_model_class3],
        [true_class3_model_class1, true_class3_model_class2, true_class3_model_class3]]
    )
  return confusion_matrix

In [287]:
MLP1 = MLP(4400, 1024, 512, 512, 128, 32, 3)

In [288]:
train(MLP1, X_train, Y_train, 500)

Score: 63.83495145631068 
Epoch:  0
Score: 73.7864077669903 
Epoch:  1
Score: 74.02912621359224 
Epoch:  2
Score: 74.02912621359224 
Epoch:  3
Score: 74.02912621359224 
Epoch:  4
Score: 74.02912621359224 
Epoch:  5
Score: 74.02912621359224 
Epoch:  6
Score: 74.02912621359224 
Epoch:  7
Score: 74.02912621359224 
Epoch:  8
Score: 74.02912621359224 
Epoch:  9
Score: 74.02912621359224 
Epoch:  10
Score: 74.02912621359224 
Epoch:  11
Score: 74.02912621359224 
Epoch:  12
Score: 74.02912621359224 
Epoch:  13
Score: 74.02912621359224 
Epoch:  14
Score: 74.02912621359224 
Epoch:  15
Score: 74.02912621359224 
Epoch:  16
Score: 74.02912621359224 
Epoch:  17
Score: 74.02912621359224 
Epoch:  18
Score: 74.02912621359224 
Epoch:  19
Score: 74.02912621359224 
Epoch:  20
Score: 74.02912621359224 
Epoch:  21
Score: 74.02912621359224 
Epoch:  22
Score: 74.02912621359224 
Epoch:  23
Score: 74.02912621359224 
Epoch:  24
Score: 74.02912621359224 
Epoch:  25
Score: 74.02912621359224 
Epoch:  26
Score: 74.02

In [284]:
print(model_accuracy(MLP1, X_dev, Y_dev))
print(confusion_matrix(MLP1, X_dev, Y_dev))

76.92307692307692
tensor([[ 0,  5,  0],
        [ 0, 40,  0],
        [ 0,  7,  0]])
